In [2]:
import pandas as pd
import gensim 
from gensim.models import Word2Vec 
from tqdm import tqdm
import re
import numpy as np
  
dataset = pd.read_csv("data/names_combined.csv")

dataset.dropna(inplace=True)
dataset.replace('b','black',inplace=True)
  
data = [] 
corpus = []

def features(name):
    name = re.sub('[^a-zA-Z]', ' ', name)
    name = re.sub('  ', ' ', name)
    name = name.lower()
    name = name[:30]
    name += '#'*(30-len(name))
    bigrams = []
    trigrams = []
    for i in range(len(name)-1):
        bigrams.append(name[i:i+2])
        if i < len(name) - 2:
            trigrams.append(name[i:i+3])
    trigrams.append(name[-3:])
    return list(name)+bigrams#+trigrams

for i in tqdm(range(len(dataset['name']))):
    temp = []
    review = dataset['name'].values[i]
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = re.sub('  ', ' ', review)
    if review[0] == ' ':
        review=review[1:]
    temp = features(review)
    data.append(temp)
    corpus.append(review)

100%|██████████| 120719/120719 [00:06<00:00, 20119.27it/s]


In [3]:
model1 = gensim.models.Word2Vec(data, min_count = 1, size = 30, window = 10) ## 300,10
print('model created')
model1.train(data,total_examples=model1.corpus_count, epochs=10)
print('model trained')

model created
model trained


In [4]:
vecs = model1[features('rajes')]#.tolist()
vecs.nbytes

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


7080

In [17]:
X = []
y = dataset.iloc[:, 2].values
for c in tqdm(corpus):
    vecs = model1[features(c)].tolist()
    X += [vecs]
X = np.array(X)

  0%|          | 0/120719 [00:00<?, ?it/s]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.
100%|██████████| 120719/120719 [01:36<00:00, 1250.29it/s]


In [18]:
from imblearn.over_sampling import SMOTE, RandomOverSampler
ros = RandomOverSampler(random_state=42)
X, y = ros.fit_resample(X.reshape((120719, 59*30)), y)

In [19]:
X = X.reshape((194580,59*30))

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
y = enc.fit_transform(y.reshape(-1,1)).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)#, random_state = 0)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, LSTM, Flatten
from keras.layers.normalization import BatchNormalization
from keras import optimizers

classifier = Sequential()

######## race classification 87% test accuracy
classifier.add(Dense(input_dim=59*30, output_dim=1024, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.2))  # Dropout overfitting
classifier.add(Dense(output_dim=256, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=128, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=128, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=64, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=32, activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.1))
classifier.add(Dense(output_dim=4, activation='softmax'))

##### gender classification model 98% test acc
# classifier1.add(Dense(input_dim=300, output_dim=256, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))  # Dropout overfitting
# classifier1.add(Dense(output_dim=128, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=128, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=64, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=64, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=32, activation='relu'))
# classifier1.add(BatchNormalization())
# classifier1.add(Dropout(0.1))
# classifier1.add(Dense(output_dim=2, activation='softmax'))


classifier.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ['accuracy'])
history = classifier.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=40, batch_size=256)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=1770, activation="relu", units=1024)`
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=256)`
  del sys.path[0]
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  app.launch_new_instance()
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipyke

Train on 155664 samples, validate on 38916 samples
Epoch 1/40
155664/155664 [==============================] - 71s 453us/step - loss: 0.7214 - acc: 0.6803 - val_loss: 0.5882 - val_acc: 0.7475
Epoch 2/40
155664/155664 [==============================] - 55s 351us/step - loss: 0.5307 - acc: 0.7718 - val_loss: 0.5353 - val_acc: 0.7771
Epoch 3/40
155664/155664 [==============================] - 55s 351us/step - loss: 0.4837 - acc: 0.7918 - val_loss: 0.4945 - val_acc: 0.7952
Epoch 4/40
155664/155664 [==============================] - 56s 358us/step - loss: 0.4572 - acc: 0.8032 - val_loss: 0.4954 - val_acc: 0.7915
Epoch 5/40
155664/155664 [==============================] - 53s 341us/step - loss: 0.4387 - acc: 0.8109 - val_loss: 0.4488 - val_acc: 0.8095
Epoch 6/40
155664/155664 [==============================] - 53s 341us/step - loss: 0.4239 - acc: 0.8174 - val_loss: 0.4780 - val_acc: 0.7994
Epoch 7/40
155648/155664 [============================>.] - ETA: 0s - loss: 0.4121 - acc: 0.8219

In [15]:
import copy
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
K.set_image_dim_ordering('th')
print(K.image_data_format())

import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

# define path to save model
model_path = './fm_cnn_BN.h5'
# prepare callbacks
callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]


#size of parameters
batch_size = 256
num_classes = 4
epochs = 30
filter_pixel=3
noise = 1
droprate=0.25

# input image dimensions
img_rows, img_cols = 59, 30

input_shape = (1, img_rows, img_cols)

#Start Neural Network
model = Sequential()
#convolution 1st layer
model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), padding="same",
                 activation='relu',
                 input_shape=input_shape)) #0
model.add(BatchNormalization())
model.add(Dropout(droprate))#3
#model.add(MaxPooling2D())

#convolution 2nd layer
model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), activation='relu',border_mode="same"))#1
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(droprate))#3

#convolution 3rd layer
model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), activation='relu',border_mode="same"))#1
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(droprate))#3

#Fully connected 1st layer
model.add(Flatten()) #7
model.add(Dense(500,use_bias=False)) #13
model.add(BatchNormalization())
model.add(Activation('relu')) #14
model.add(Dropout(droprate))      #15

#Fully connected final layer
model.add(Dense(4)) #8
model.add(Activation('softmax')) #9


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.RMSprop(),
              metrics=['accuracy'])

model.summary()

#Save Model=ON
history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test),shuffle=True,callbacks=callbacks)

score = model.evaluate(X_test, y_test, verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

channels_first


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same")`
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:64: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=(3, 3), activation="relu", padding="same")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 64, 59, 30)        640       
_________________________________________________________________
batch_normalization_9 (Batch (None, 64, 59, 30)        120       
_________________________________________________________________
dropout_9 (Dropout)          (None, 64, 59, 30)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 64, 59, 30)        36928     
_________________________________________________________________
batch_normalization_10 (Batc (None, 64, 59, 30)        120       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 29, 15)        0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 64, 29, 15)        0         
__________

KeyboardInterrupt: 